# Homework for Week 9

In [2]:
import numpy as np
import onnxruntime as rt
from io import BytesIO
from urllib import request
import os
from PIL import Image

In [3]:
# Download Model Files
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
MODEL_FILE = "hair_classifier_v1.onnx"
DATA_FILE = "hair_classifier_v1.onnx.data"

if not os.path.exists(MODEL_FILE):
    request.urlretrieve(f"{PREFIX}/{MODEL_FILE}", MODEL_FILE)

if not os.path.exists(DATA_FILE):
    request.urlretrieve(f"{PREFIX}/{DATA_FILE}", DATA_FILE)

print("Model files ready")

Model files ready


In [4]:
# Q1 - Output Node Name
session = rt.InferenceSession(MODEL_FILE)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

output_name

'output'

In [5]:
# Q2 - Target Image Size
input_shape = session.get_inputs()[0].shape
target_size = (input_shape[2], input_shape[3])

target_size

(200, 200)

In [6]:
# Helper Functions
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [7]:
# Q3 - First Pixel Value
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, target_size)

# Convert to array and normalize
x = np.array(img, dtype='float32')
x = x / 255.0

# ImageNet normalization
mean = np.array([0.485, 0.456, 0.406], dtype='float32')
std = np.array([0.229, 0.224, 0.225], dtype='float32')
x = (x - mean) / std

# Reshape for model
X = np.array([x])
X = X.transpose(0, 3, 1, 2)

X[0, 0, 0, 0]

np.float32(-1.073294)

In [8]:
# Q4 - Model Output
preds = session.run([output_name], {input_name: X})
preds[0][0][0]

np.float32(0.09156641)